In [15]:
import pandas as pd
import numpy as np
import pyodbc
import datetime
pd.set_option('display.float_format', lambda x: '%.3f' % x)

labs = ['AID','Ilex','Electra','MH']
bill_types = ['Sampler','Lab']

class Price:
    def __init__(self,lab,test_type,price):
        self.lab = lab
        self.test_type = test_type
        self.price = price

    def to_dict(self):
        return {(self.lab ,self.test_type):self.price}


lab_prices = {}
#AID
lab_prices.update(Price(lab = 'AID',test_type='single_regular', price = 119.34).to_dict())
lab_prices.update(Price(lab = 'AID',test_type='son_no_bag', price = 15.21).to_dict())
lab_prices.update(Price(lab = 'AID',test_type='son_with_bag', price = 17.55).to_dict())
lab_prices.update(Price(lab = 'AID',test_type='Father_pooling', price = 119.34).to_dict())
#Ilex
lab_prices.update(Price(lab = 'Ilex',test_type='single_regular', price = 119.34).to_dict())
lab_prices.update(Price(lab = 'Ilex',test_type='son_no_bag', price = 29.25).to_dict())
lab_prices.update(Price(lab = 'Ilex',test_type='son_with_bag', price = 17.55).to_dict())
lab_prices.update(Price(lab = 'Ilex',test_type='Father_pooling',price = 159.12).to_dict())
#Electra
lab_prices.update(Price(lab = 'Electra',test_type='single_regular', price = 89.622).to_dict())
lab_prices.update(Price(lab = 'Electra',test_type='son_no_bag', price = 25.74).to_dict())
lab_prices.update(Price(lab = 'Electra',test_type='son_with_bag', price = 23.985).to_dict())
lab_prices.update(Price(lab = 'Electra',test_type='Father_pooling', price = 114.66).to_dict())
lab_prices.update(Price(lab = 'Electra',test_type='single_pooling', price = 71.6976).to_dict())
#MH
lab_prices.update(Price(lab = 'MH',test_type='single_regular', price = 116.5203).to_dict())
lab_prices.update(Price(lab = 'MH',test_type='son_no_bag', price = 28.08).to_dict())
lab_prices.update(Price(lab = 'MH',test_type='son_with_bag', price = 17.55).to_dict())
lab_prices.update(Price(lab = 'MH',test_type='Father_pooling', price = 116.5203).to_dict())

def get_lab_price(lab,test_type):
    if (lab,test_type) in lab_prices.keys():
        return lab_prices[(lab,test_type)]
    else:
        return -10000000
    
lab_prices



{('AID', 'single_regular'): 119.34,
 ('AID', 'son_no_bag'): 15.21,
 ('AID', 'son_with_bag'): 17.55,
 ('AID', 'Father_pooling'): 119.34,
 ('Ilex', 'single_regular'): 119.34,
 ('Ilex', 'son_no_bag'): 29.25,
 ('Ilex', 'son_with_bag'): 17.55,
 ('Ilex', 'Father_pooling'): 159.12,
 ('Electra', 'single_regular'): 89.622,
 ('Electra', 'son_no_bag'): 25.74,
 ('Electra', 'son_with_bag'): 23.985,
 ('Electra', 'Father_pooling'): 114.66,
 ('Electra', 'single_pooling'): 71.6976,
 ('MH', 'single_regular'): 116.5203,
 ('MH', 'son_no_bag'): 28.08,
 ('MH', 'son_with_bag'): 17.55,
 ('MH', 'Father_pooling'): 116.5203}

In [20]:
import pandas as pd
df = pd.read_csv(r"C:\Users\eilon.eilstein\Downloads\AID raw.csv")
df.head(2)

,barcode,arrival_date,result_date,result,??? ?????,type
0,65871994.000,2021-07-29 00:17:19,2021-07-29 08:00:52,Negative,????,singel_regular
1,65873876.000,2021-07-29 14:26:17,2021-07-29 18:58:27,Negative,????,singel_regular


In [22]:
pd.options.display.float_format = '{:,.2f}'.format

def get_Count_df(df):
    df = df.copy(deep=True)
    df = df.groupby('type').count().reset_index()[['type', 'barcode']]
    return df


def main(df):
    return get_Count_df(df)

df = main(df)
print(df)

df['Price'] = df.apply(lambda row: get_lab_price('AID', row['type']), axis=1)
df

             type  barcode
0  Father_pooling        1
1  singel_regular        1
2      son_no_bag        1


,type,barcode,Price
0,Father_pooling,1,119.34
1,singel_regular,1,"-10,000,000.00"
2,son_no_bag,1,15.21


In [24]:
lab_prices


{('AID', 'single_regular'): 119.34,
 ('AID', 'son_no_bag'): 15.21,
 ('AID', 'son_with_bag'): 17.55,
 ('AID', 'Father_pooling'): 119.34,
 ('Ilex', 'single_regular'): 119.34,
 ('Ilex', 'son_no_bag'): 29.25,
 ('Ilex', 'son_with_bag'): 17.55,
 ('Ilex', 'Father_pooling'): 159.12,
 ('Electra', 'single_regular'): 89.622,
 ('Electra', 'son_no_bag'): 25.74,
 ('Electra', 'son_with_bag'): 23.985,
 ('Electra', 'Father_pooling'): 114.66,
 ('Electra', 'single_pooling'): 71.6976,
 ('MH', 'single_regular'): 116.5203,
 ('MH', 'son_no_bag'): 28.08,
 ('MH', 'son_with_bag'): 17.55,
 ('MH', 'Father_pooling'): 116.5203}

# Lab Functions

In [16]:
def drop_labs_tests(df,operator_name):
    df = df.sort_values(by=['type']).drop_duplicates(subset=['barcode'])
    df['freq'] = df.groupby('barcode')['barcode'].transform('count')
    return df

def get_receipt(df):
    receipt = df.groupby('type').sum()
    return receipt

# Main Part 

In [17]:
bill_type = 'Lab' ## Sampler/Lab
operator_name = 'AID'
# file_name = 'july files\AID\AID raw.xlsx'
file_name = 'bills_sample.xlsx'
global df

def the_main():
    df = pd.read_excel(file_name)
    df = drop_tests(df, bill_type, operator_name)  
    df = get_Price_column(df, bill_type, operator_name)
    df = get_sla_column(df, bill_type, operator_name)
    df = found_on_bi(df, bill_type, operator_name)
    return df

    return 0


def drop_tests(df,bill_type,operator_name):
    df = df.copy(deep=True)
    if bill_type == 'Lab':
        df = drop_labs_tests(df,operator_name)
#         print(df)
        return df
    elif bill_type == 'Sampler':
        return df
    else:
        return -100
        
def get_Price_column(df,bill_type,operator_name):
    df = df.copy(deep=True)
    if bill_type == 'Lab':
        df['result_date'].fillna(df['arrival_date'], inplace = True)
        df['Price'] = df.apply(lambda row: get_lab_price(operator_name,row['type']), axis=1)
        print(df)
        return df
    elif bill_type == 'Sampler':
        return df
    else:
        return -100
        
def get_sla_column(df,bill_type,operator_name):
    df = df.copy(deep=True)
    if bill_type == 'Lab':
        return df
    elif bill_type == 'Sampler':
        return df
    else:
        return -100

def found_on_bi(df,bill_type,operator_name):
    df = df.copy(deep=True)
    if bill_type == 'Lab':
        return df
    elif bill_type == 'Sampler':
        return df
    else:
        return -100

df = the_main()
df

     barcode        arrival_date         result_date         result  \
0  309031186 2021-07-14 20:37:51 2021-07-15 01:38:52       Negative   
1  309026886 2021-07-09 04:06:01 2021-07-09 11:06:15       Negative   
2  309026885 2021-07-08 19:48:55 2021-07-09 01:55:23       Negative   
3  309026884 2021-07-07 09:09:56 2021-07-07 17:04:53       Negative   
4  309026883 2021-07-07 09:11:16 2021-07-07 17:04:57  Weak Positive   

     סוג מבחנה            type  freq   Price  
0  מבחנת איגום  Father_pooling     1 119.340  
1  מבחנת איגום  Father_pooling     1 119.340  
2  מבחנת איגום  Father_pooling     1 119.340  
3  מבחנת איגום  Father_pooling     1 119.340  
4  מבחנת איגום  Father_pooling     1 119.340  


,barcode,arrival_date,result_date,result,סוג מבחנה,type,freq,Price
0,309031186,2021-07-14 20:37:51,2021-07-15 01:38:52,Negative,מבחנת איגום,Father_pooling,1,119.340
1,309026886,2021-07-09 04:06:01,2021-07-09 11:06:15,Negative,מבחנת איגום,Father_pooling,1,119.340
2,309026885,2021-07-08 19:48:55,2021-07-09 01:55:23,Negative,מבחנת איגום,Father_pooling,1,119.340
3,309026884,2021-07-07 09:09:56,2021-07-07 17:04:53,Negative,מבחנת איגום,Father_pooling,1,119.340
4,309026883,2021-07-07 09:11:16,2021-07-07 17:04:57,Weak Positive,מבחנת איגום,Father_pooling,1,119.340


In [11]:
df[df['freq']>1].sort_values(by=['type']).drop_duplicates(subset=['barcode'])

,barcode,arrival_date,result_date,result,סוג מבחנה,type,freq,Price


In [77]:
df.head()

,barcode,arrival_date,result_date,result,סוג מבחנה,type,freq,Price
195458,309031186,2021-07-14 20:37:51,2021-07-15 01:38:52,Negative,מבחנת איגום,Father_pooling,1,119.340
194429,309026886,2021-07-09 04:06:01,2021-07-09 11:06:15,Negative,מבחנת איגום,Father_pooling,1,119.340
194428,309026885,2021-07-08 19:48:55,2021-07-09 01:55:23,Negative,מבחנת איגום,Father_pooling,1,119.340
194427,309026884,2021-07-07 09:09:56,2021-07-07 17:04:53,Negative,מבחנת איגום,Father_pooling,1,119.340
194426,309026883,2021-07-07 09:11:16,2021-07-07 17:04:57,Weak Positive,מבחנת איגום,Father_pooling,1,119.340


In [81]:
get_recipt(df)[['freq','Price']]

,freq,Price
type,,
Father_pooling,11653,1390669.020
singel_regular,145131,17319933.540
son_no_bag,186021,2829379.410


In [8]:
df

NameError: name 'df' is not defined

(342805, 7)

In [12]:
#הוספת שדה כפילויות
df['freq'] = df.groupby('ברקוד')['ברקוד'].transform('count')

In [19]:
df[df['freq']>1].groupby(['תוצאה']).count()

,ברקוד,קוד AID,גורם שולח,מועד קליטה,מועד תשובה,סוג מבחנה,freq
תוצאה,,,,,,,
Negative,19031,19031,19031,19031,19031,19031,19031
Performed Inconclusive,120,120,120,120,120,120,120
Positive,870,870,870,870,870,870,870
Weak Positive,57,57,57,57,57,57,57


ModuleNotFoundError: No module named 'pandas_to_sql'

In [22]:
import streamlit

ModuleNotFoundError: No module named 'streamlit'